## Cookies
Cookies in a Spring web application are a common way to store data on the client side and send it back to the server with each request. They are often used for session management, authentication tokens, user preferences, and other client-specific data.

## Cookie usage in Spring Web Application

### Setting a Cookie
To set a cookie in a Spring web application, you can add a cookie to the `HttpServletResponse` object.

```
import javax.servlet.http.Cookie;
import javax.servlet.http.HttpServletResponse;

@RestController
public class Controller {

    @GetMapping("/home")
    public String setCookie(HttpServletResponse response) {
        Cookie cookie = new Cookie("myCookie", value);
        cookie.setPath("/");
        cookie.setHttpOnly(true); // Secure the cookie to prevent client-side scripts from accessing it
        cooke.setMaxAge(7 * 4 * 60 * 60); // Set the cookie to expire in 7 days

        response.addCookie(cookie);

        return "Cookie set";
    }
}
```

### Retrieving a Cookie
To retrieve a cookie in a Spring web application, you can access the cookies from the `HttpServletRequest` object.
```
import javax.servlet.http.Cookie;
import javax.servlet.http.HttpServletRequest;

@RestController
@RequestMapping("/hello")
public class OtherController {

    @GetMapping("/world")
    public String helloWorld(HttpServletRequest request) {
        Cookie[] cookies = request.getCookies();
        Cookie myCookie = Arrays.stream(cookies)
                                .filter(cookie -> "myCookie".equals(cookie.getName()))
                                .findFirst()
                                .map(Cookie::getValue)
                                .orElse("Cookie not found");
        return "Hello World";
    }
}
```

### Deleting a Cookie
To delete a cookie, you need to set its maximum value to 0 and add it to the response.
```
import javax.servlet.http.Cookie;
import javax.servlet.http.HttpServletResponse;

@RestController
public class Controller {

    @GetMapping("/bye")
    public String deleteCookie(HttpServletResponse response) {
        Cookie cookie = new Cookie("myCookie", null);
        cookie.setPath("/");
        cooke.setMaxAge(0); // Set the cookie to expire immediately

        response.addCookie(cookie);

        return "Bye";
    }
}
```

### @CookieValue
The `@CookieValue` annotation in Spring MVC is sued to bind the value of an HTTP cookie to a method parameter in a controller. This makes it easy to work with cookies in your request handling methods.

#### Reading a Cookie
You can use the `@CookieValue` annotation to bind the value of a cookie to a pethod parameter.
```
import org.springframework.web.bind.annotation.CookieValue;

@RestController
public class CookieController {

    @GetMapping("/read-cookie")
    public String readCookie(@CookieValue(value = "my Cookie", defaultValue = "default") String myCookie) {
        return "Cookie value: " + myCookie;
    }
}
```

By default, the `@CookieValue` annotation is required, meaning that the cookie must be present. If it's not, an exception will be thrown. You can make the cookie value optional by setting the `required` attribute to `false`. This will set the method parameter to `null` if the cookie is not present.
```
import org.springframework.web.bind.annotation.CookieValue;

@RestController
public class CookieController {

    @GetMapping("/read-cookie-optional")
    public String readCookieOptional(@CookieValue(value = "myCookie", required = false) String myCookie) {
        return "Cookie value: " + (myCookie != null ? myCookie : "No cookie found");
    }
}
```

# Bean lifecycle
In Spring, the concept of scope determines hte lifecycle and visibility of beans.
- Request Scope
- Session Scope
- 

## Request Scope
Request scope is a specific scope for beans that are created and exist only within the lifecycle of a single HTTP request. This means that a new instance of a request-scoped bean is created for each HTTP request, and that instance is discarded once the request was processed.

### Purpose of Request Scope
- Statelessness: When you need a bean to be stateless and independent for each HTTP request.
- Request-specific Data: When the bean holds request-specific data that should not be shared between requests.
- Thread Safety: Request-scoped beans are inherently thread-safe since each request gets its own instance.

### Defining Request Scope
To define a bean with request scope, you can use the `@Scope` annotation with the `ConfigurableBeanFactory.SCOPE_REQUEST` constant.

1. Defining the Bean

```
@Component
@Scope(ConfigurableBeanFactory.SCOPE_REQUEST)
public class RequestScopedBean {
    private String requestData;

    public String getRequestData() {
        return requestData;
    }

    public void setRequestData(String requestData) {
        this.requestData = requestData;
    }
}
```
2. Useing the Bean in a Controller
```
@RestController
@RequestMapping("/example")
public class ExampleController {

    @Autowired
    private RequestScopedBean requestScopedBean;

    @GetMapping("/set")
    public String setRequestData() {
        requestScopedBean.setRequestData("Sample Data");
        return "Data set in request scope";
    }

    @GetMapping("/get")
    public String getRequestData() {
        return "Data from request scope: " + requestScopedBean.getRequestData();
    }
}
```

# Session
Because HTTP is a stateless protocol, meaning that each request from a client to a server is independent and should not retain any information about previous interactions. Sessions bridge this gab by maintaining state across multiple requests, allowing the server to recognize users and remember their interactions.

Sessions enable web application to keep track of logged-in users, maintaining their authentication state throughout their visit. This persistent state ensures taht users do not need to reauthorize with every new request, thus streamlining user expereience.

It is more seucire using sessions, because it is a secure way to handler user data without exposign sensitive information on the client-side. Since session data is stored on the server, it reduces the risk of data being tampered with or accessed by unauthorized third parties.

## JSESSIONID
JSESSIONID is a unique identifier for a user session. It is stored in a cookie. It allws the server to link a user's HTTP requests to their session data on the server.

When  session is created, the server generates a `JSESSIONIID` and sends it to the client as a cookie. The client sends this cookies with subsequent requests, allowing the server to maintain user state across multiple requests.

The JSESSIONID is generated by servlet container, different servlet containers have different algorithms.

By default it is managed by serverlet ontainer and stored in memory as a `HashMap`.


## Cookies vs Sessions
Cookies
- Client-Side Storage: Cookes are small pieces of data stored on the client-side (browser).
- State Management: They are used to maintain state between HTTP request since HTTP is stateless.
- Usage: Commonly used for storing user preferences, authentication tokens, etc.
- Limitations: Since they are stored on the client-side, they can be manipulated and have size limitations.

Sessions
- Server-Side Storage: Sessions are stored on the server0side and hold information about a user across multiple HTTP requests.
- State Management: A session is typically identified by a unique identifier (session ID) which is stored in a cookie (`JSESSIONID`) or URL.
- Usage: Used for storing user-specific data that should not be exposed to the client, such as login status, suer preferencs, etc.
- Challanges: In a server cluster, sharing session information cna be challanging because session data needs t oe accessible across differetn servers.

## Session Scope
Session scope in Spring are for beans with lifecycles that are created and exist for the duration of an HTTP session. This means that a new instance of a session-scoped bean is created for each user session-scoped bean is created for each user session and is shared across multiple requests within that session.

When to Use Session Scope
- User-specific data: When you need to store user-specific data that should persist across multiple requests in a single session.
- State Management: When you need to maintain state information for a user during their session.
- Thread Safety: Session-scoped beans are inherently thread-safe within the scope of a single user session.

### Defining a scope with @Scope(WebApplicationContext.SCOPE_SESSION)
You can define a session-scoped bean by using `@Scope` annotation with the value `WebApplicationContext.SCOPE_SESSION`. this can be used in conjunction with component-scanning annotations like `@Component`, `@Controller`, `@Service`, `@Repository`, or `@Bean`

Example in conjunction with `@Component`
```
@Component
@Scope(WebApplicationContext.SCOPE_SESSION)
public class SessionScopedBean {
    private String data;

    // Getters and setters
    public String getData() {
        return data;
    }

    public void setData(String data) {
        this.data = data;
    }
}
```

Example in conjuntion with `@Bean` in a Configuration Class
```
@Configuration
public class AppConfig {

    @Bean
    @Scope(WebApplicationContext.SCOPE_SESSION)
    public SessionScopedBean sessionScopedBean() {
        return new SessionScopedBean();
    }
}
```

### Accessing Session-scoped Beans
Session-scoped beans can be injected into other Spring beans, When injected into singleton-scoped beans, you typically use proxy mode to ensure that the correct instance is injected for each session.

Example with Proxy Mode
```
@Service
public class MyService {

    private final SessionScopedBean sessionScopedBean;

    @Autowired
    public MyService(SessionScopedBean sessionScopedBean) {
        this.sessionScopedBean = sessionScopedBean;
    }

    public void performService() {
        // Use the sessionScopedBean
    }
}
```

Example of Usage in a Controller
```
@RestController
public class MyController {

    private final SessionScopedBean sessionScopedBean;

    @Autowired
    public MyController(SessionScopedBean sessionScopedBean) {
        this.sessionScopedBean = sessionScopedBean;
    }

    @PostMapping("/set-session-data")
    public String setSessionData(@RequestParam String data) {
        sessionScopedBean.setData(data);
        return "Session data set";
    }

    @GetMapping("/get-session-data")
    public String getSessionData() {
        return "Session data: " + sessionScopedBean.getData();
    }
}
```

## Conversational Scope
Conversation scope  not a built-in scope in Spring, it refers to managing state across a series of interactions (or conversation) within a user's session. This concept is often implemented using session scoped to store data that needs to persist acrosss multiple requests but not necessarily the entire session.

In Spring MVC, conversational scope can be managed using `@SessionAttributes` to handle attributes that need to persist for the duration of a conversation (e.g., a multi-step form process).


### @SessionAttributes
The `@SessionAttributes` annotation is used to store model attributes in the HTTP session across multiple requests. It is useful for web applications that require some attributes to persist across multiple requests.

How `@SessionAttributes` Works
1. Declaration: Use `@SessionAttributes` at the class level in a Spring MVC controller to specify which model attributes should be stored in the session.
2. Storage: The specified attributes are stored in the HTTP session when they are first added to the model.
3. Access and Update: These attributes can be accessed and updated across multiple requests.
4. Removal: Use `SessionStatus.setComplete()` to remove the attributes from the session once the conversation is complete.

#### Example Use Case: Multiple-Step Form
Controller with `@SessionAttributes`
```
@Controller
@SessionAttributes("userForm")
public class UserFormController {

    @GetMapping("/startForm")
    public String startForm(Model model) {
        UserForm userForm = new UserForm();
        model.addAttribute("userForm", userForm); // Add the form object to the model
        return "formStep1"; // Return the view name for step 1
    }

    @PostMapping("/formStep1")
    public String handleStep1(@ModelAttribute("userForm") UserForm userForm) {
        // Process step 1 data
        return "formStep2"; // Move to step 2
    }

    @PostMapping("/formStep2")
    public String handleStep2(@ModelAttribute("userForm") UserForm userForm) {
        // Process step 2 data
        return "formStep3"; // Move to step 3
    }

    @PostMapping("/completeForm")
    public String completeForm(@ModelAttribute("userForm") Userform userform, SessionStatus sessionStatus)
        // Process final step data
        sessionStatus.setComplete(); // Remove the userForm attribute from the session
        return "fromCompleted";
    }
}
```

Flow
1. Controller Class: The `UserFormController` class is annotated with `@SessionAttributes("userForm")`, indicating thatthe `userForm` attribute should be stored in the session.
2. startForm Mthod: Initializes a new `UserForm` object and adds it to the model. Because of the `@SessionAttributes` annotation, this object will be stored in the session.
3. handleStep1 and handleStep2 Methods: Handle the data from the respoective form steps and proceed to the next step. The `UserForm` object is retrieved from the session and updated with each step.
4. completeForm Method: Processes teh final step, completes the form submission, and calls `sessionStatus.setComplete()` to remove the `userForm` attribute from the session, ending the conversation.

Example Use Case: User Registration<br>
Controller with `@SessionAttributes`
```
@Controller
@SessionAttributes("registration")
public class RegistrationController {

    @GetMapping("/register")
    public String showRegistrationForm(Model model) {
        Registration registration = new Registration();
        model.addAttribute("registration", registration);
        return "registrationStep1";
    }

    @PostMapping("/registrationStep1")
    public String processStep1(@ModelAttribute("registration") Registration registration) {
        // Process step 1
        return "registrationStep2";
    }

    @PostMapping("/registrationStep2")
    public String processStep2(@ModelAttribute("registration") Registration registration) {
        // Process step 2
        return "registrationStep3";
    }

    @PostMapping("/completeRegistration")
    public String completeRegistration(@ModelAttribute("registration") Registration registration, SessionStatus sessionStatus) {
        // Finalize registration
        sessionStatus.setComplete();
        return "registrationCompleted";
    }
}
```

## @SessionScope vs @SessionAttributes
1. Scope Duration:
- - `@SessionScope`: Bean persists for the entire session.
  - `@SessionAttributes`: Attributes persist only for the duration of the conversation.

2. Usage Context:
- - `@SessionScope`: Used for defining session-scope beans in any Spring component.
  - `@SessionAttributes`: Used specifically within Spring MVC controllers to manage model attributes.

3. Lifecycle Management:
- - `@SessionScope`: Managed by Spring's bean lifecycle, tied to the session lifecycle.
  - `@SessionAttributes`: Managed by Spring MVC, tied to the controller's conversational flow.

4. Purpose:
- - `@SessionScope`: General session-wide state management.
  - `@SessionAttributes`: Specific to multi-step interactions and conversational state withing a controller.

## HttpSession
The `HttpSession` interface in Java Servlets is used to store session data between requtest from the same user. 

How to work with HttpSession
1. Creating or Retrieving a Session
```
HttpSession session = request.getSession();
```
- - reqeust.getSession(): This method either retrieves the current session associated with the user or creates a new session if one does not exist. The session persists until it is explicitly invalidated or times out.
 
2. Storing Data in the Session
```
session.setAttribute("key", value);
```
- - session.setAttribute(String key, Object value): This method stores data in the session under a specific key. The data can be of any `Object` type.

3. Removing a session
```
session.invalidate();
```
Session and its attrivutes persist across multiple reqtuests from the same client. The session remains active until it temes out (based on server settings) or is invalidated using `session.invalidate()`.

webclient
openfeign, declarative, copy the controller

webclient openfeign vs resttemplage

, spring cloud load balancer, discovery service (registration) (eureka)
server side discovery
- nginx
- aws elb
client side discovery (dns)
- Netflix eureka (eureka server) eureka client
- zookeeper
- consul


discoveryClient to get ip to make call to another service

loadbalancing
loadbalancerclient (client side load balancer)
- downside, loadbalancer for every server (no distributed load balancer)